In [ ]:
# Import required libraries
from rdflib import Graph
from ontopy import get_ontology

units = get_ontology("https://w3id.org/emmo/disciplines/units").load()


In [ ]:
battery = get_ontology("https://w3id.org/emmo/domain/battery").load()
battery.Battery
battery.Device

In [ ]:
from owlready2 import Thing, ThingClass

def safe_set_label(entity, label):
    """Safely set a label for an entity."""
    if isinstance(label, str):
        entity.label = label
    else:
        print(f"Invalid label for {entity}: {label}")

def MergeOntologies(ontology1, ontology2):
    """
    Merge two ontologies, adding missing classes, individuals, and properties 
    from ontology1 to ontology2 while avoiding duplicates.
    """
    # Add missing classes
    for cls1 in ontology1.classes():
        if cls1.name not in {cls2.name for cls2 in ontology2.classes()}:
            valid_parents = [
                parent for parent in cls1.is_a if isinstance(parent, ThingClass)
            ]
            if not valid_parents:
                valid_parents = [Thing]  # Default to Thing if no valid parents
            
            # Add class to ontology2
            new_class = ontology2.new_class(cls1.name, valid_parents)
            safe_set_label(new_class, cls1.name)

    # Add missing individuals
    for ind1 in ontology1.individuals():
        if ind1.name not in {ind2.name for ind2 in ontology2.individuals()}:
            # Create a new individual in ontology2 with the same properties
            new_ind = ontology2.new_entity(ind1.name, ind1.is_a)
            try:
                for prop in ind1.get_properties():
                    values = prop[ind1]  # Get the values for this property
                    for value in values:
                        new_ind[prop] = value
            except Exception as e:
                print(f"Failed to copy property for individual {ind1.name}: {e}")

    # Add missing properties
    for prop1 in ontology1.properties():
        if prop1.name not in {prop2.name for prop2 in ontology2.properties()}:
            try:
                new_property = ontology2.new_entity(prop1.name, ThingClass)
                if prop1.domain:
                    new_property.domain = prop1.domain
                if prop1.range:
                    new_property.range = prop1.range
            except Exception as e:
                print(f"Failed to add property {prop1.name}: {e}")

    return ontology2






merged = MergeOntologies(battery, units)


Failed to copy property for individual 0000-0001-8869-3718: Invalid label definition, must be a string: 'schema.org.name'
Failed to copy property for individual 0000-0002-1560-809X: Invalid label definition, must be a string: 'schema.org.name'
Failed to copy property for individual 0000-0002-2547-155X: Invalid label definition, must be a string: 'schema.org.name'
Failed to copy property for individual 0000-0002-4181-2852: Invalid label definition, must be a string: 'core.prefLabel'
Failed to copy property for individual 0000-0002-5174-8508: Invalid label definition, must be a string: 'core.prefLabel'
Failed to copy property for individual 0000-0002-9668-6961: Invalid label definition, must be a string: 'core.prefLabel'
Failed to copy property for individual 0000-0002-8758-6109: Invalid label definition, must be a string: 'schema.org.name'
Failed to copy property for individual 0000-0003-2954-1233: Invalid label definition, must be a string: 'schema.org.name'
Failed to copy property for

In [ ]:
merged.Abampere

emmo.Abampere